In [ ]:
import optuna
from matplotlib import pyplot as plt
import math

USE_POSTGRES = True  # Set to True to use PostgreSQL, False for SQLite
POSTGRES_CONFIG = {
    'host': '100.107.250.125',
    'port': 5432,
    'database': 'optuna_db',
    'user': 'optuna_user',
    'password': 'optuna_password'
}

def get_valid_studies(storage_name, compressor_model):
    studies = optuna.get_all_study_names(storage=storage_name)
    valid_studies = []
    for study in studies:
        if study.startswith(f"RunIn_") and study.endswith(f"_{compressor_model}"):
            valid_studies.append(study)
    return valid_studies

def plot_pareto_studies(studies, storage_name):
    fig, ax = plt.subplots(figsize=(8, 6))
    colors = plt.cm.get_cmap('tab10', len(studies))

    for idx, study_name in enumerate(studies):
        study = optuna.load_study(study_name=study_name, storage=storage_name)
        pareto_trials = study.best_trials

        # Assume two objectives: values[0] and values[1]
        pareto_x = [t.values[0] for t in pareto_trials]
        pareto_y = [t.values[1] for t in pareto_trials]

        # Order pareto_x and pareto_y by pareto_x
        ordered = sorted(zip(pareto_x, pareto_y))
        pareto_x, pareto_y = zip(*ordered) if ordered else ([], [])

        ax.plot(pareto_x, pareto_y, marker='o', linestyle='-', color=colors(idx), label=study_name)

    ax.set_xlabel("MCC")
    ax.set_ylabel("% labeled")
    ax.set_title("Pareto Fronts for All Studies")
    ax.legend()

    plt.tight_layout()
    plt.show()

# Get database URL (SQLite or PostgreSQL)
storage_name = (f"postgresql://{POSTGRES_CONFIG['user']}:{POSTGRES_CONFIG['password']}"
                f"@{POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}"
                f"/{POSTGRES_CONFIG['database']}")      

def plot_by_compressor(compressor_model):
    studies = get_valid_studies(storage_name, compressor_model)
    if not studies:
        print(f"No valid studies found for compressor model: {compressor_model}")
        return
    plot_pareto_studies(studies, storage_name)  

/home/thalerg/anaconda3/envs/RunIn_SemiSuperv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plot_by_compressor("a")

OperationalError: (psycopg2.OperationalError) connection to server at "150.162.167.122", port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [10]:
plot_by_compressor("b")

No valid studies found for compressor model: b
